<a href="https://colab.research.google.com/github/Tan-witya/stat/blob/stat.DS_02/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [5]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'no': 0 , 'yes': 1 })

# Drop 'duration' due to data leakage
dd = df.drop('duration', axis=1)



# Define features (X) and target (y)
y = df['y_new']
X = df.drop(['y', 'y_new'], axis=1)


# Split the data BEFORE any transformations
X_train, X_test ,  y_train , y_test = train_test_split(X,y, test_size=0.3)

# Print data shape
print("shape of X:", X.shape)
print("shape of y:", y.shape)

shape of X: (41188, 15)
shape of y: (41188,)


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [6]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data


X_train[num_cols_demo]  = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo]  = scaler.fit_transform(X_test[num_cols_demo])

Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [7]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(X_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(X_train[num_cols_demo], columns=num_cols_demo, index=X_train.index).head())

print("\nMean of scaled 'age' (train):", X_train['age'].mean())
print("Std Dev of scaled 'campaign' (train):", X_train[num_cols_demo]['campaign'].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
33167  0.856967  0.524717
23119 -1.058172  1.254081
28853 -1.632714 -0.569329
24158 -0.004846 -0.569329
15989 -0.292117  2.348127

Scaled X_train 'age' and 'campaign' head:
            age  campaign
33167  0.856967  0.524717
23119 -1.058172  1.254081
28853 -1.632714 -0.569329
24158 -0.004846 -0.569329
15989 -0.292117  2.348127

Mean of scaled 'age' (train): -9.85803802518262e-19
Std Dev of scaled 'campaign' (train): 1.0000173428951202


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [8]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')



X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

- **Ordinal Encoding**

In [9]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [10]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train[ord_col_demo] =  ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] =  ordinal_encoder.fit_transform(X_test[ord_col_demo])



Let's take a look at the imputed and ordinal-encoded `education`.

In [11]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train[ord_col_demo], columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train[ord_col_demo], columns=ord_col_demo, index=X_train.index).iloc[20:25])


Original X_train 'education' head:
       education
32082        1.0
29122        6.0
15257        1.0
40999        1.0
26451        4.0

Imputed X_train 'education' head (after imputer.transform):
       education
32082        1.0
29122        6.0
15257        1.0
40999        1.0
26451        4.0

Ordinal Encoded X_train 'education' head:
       education
32082        1.0
29122        6.0
15257        1.0
40999        1.0
26451        4.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [12]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo]= imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.fit_transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [13]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here
X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot  = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 11)

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [14]:
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [15]:
numerical_features = ['age','campaign','pdays','previous']
ordinal_features = ['education']
nominal_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [16]:
X_train_copy.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33167,0.856967,blue-collar,married,2.0,NaN,yes,no,cellular,may,tue,85,0.524717,999,1,failure
23119,-1.058172,technician,single,4.0,no,yes,no,cellular,aug,tue,130,1.254081,999,0,nonexistent
28853,-1.632714,blue-collar,single,1.0,no,yes,no,telephone,apr,thu,287,-0.569329,999,0,nonexistent
24158,-0.004846,technician,married,5.0,no,no,no,cellular,nov,mon,550,-0.569329,999,0,nonexistent
15989,-0.292117,management,married,6.0,no,yes,no,cellular,jul,tue,70,2.348127,999,0,nonexistent


#1.)numerical_features


In [17]:
scaler = StandardScaler()

X_train_copy[numerical_features] = scaler.fit_transform(X_train_copy[numerical_features])
X_test_copy[numerical_features] = scaler.transform(X_test_copy[numerical_features])

In [18]:
job_one_columns = [col for col in X_train_copy.columns if 'job_' in col]


X_train_copy = X_train_copy.drop(columns=job_one_columns)
X_test_copy = X_test_copy.drop(columns=job_one_columns)

X_train_copy

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
33167,0.856967,blue-collar,married,2.0,NaN,yes,no,cellular,may,tue,85,0.524717,0.195177,1.646463,failure
23119,-1.058172,technician,single,4.0,no,yes,no,cellular,aug,tue,130,1.254081,0.195177,-0.349802,nonexistent
28853,-1.632714,blue-collar,single,1.0,no,yes,no,telephone,apr,thu,287,-0.569329,0.195177,-0.349802,nonexistent
24158,-0.004846,technician,married,5.0,no,no,no,cellular,nov,mon,550,-0.569329,0.195177,-0.349802,nonexistent
15989,-0.292117,management,married,6.0,no,yes,no,cellular,jul,tue,70,2.348127,0.195177,-0.349802,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25246,0.569696,unemployed,married,4.0,NaN,yes,no,cellular,nov,tue,81,-0.204647,0.195177,-0.349802,nonexistent
16607,-1.058172,technician,single,6.0,no,yes,no,cellular,jul,wed,438,-0.569329,0.195177,-0.349802,nonexistent
32856,-0.866658,technician,married,6.0,no,yes,no,cellular,may,mon,282,-0.569329,0.195177,-0.349802,nonexistent
21204,-0.196360,technician,divorced,6.0,NaN,yes,no,cellular,aug,mon,67,-0.204647,0.195177,-0.349802,nonexistent


In [19]:
imputer_nom = SimpleImputer(strategy='most_frequent')
X_train_copy[nominal_features] = imputer_nom.fit_transform(X_train_copy[nominal_features])
X_test_copy[nominal_features] = imputer_nom.transform(X_test_copy[nominal_features])



In [20]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


X_train_onehot = onehot_encoder.fit_transform(X_train_copy[nominal_features])
X_test_onehot = onehot_encoder.transform(X_test_copy[nominal_features])


onehot_feature_names = onehot_encoder.get_feature_names_out(nominal_features)


X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=onehot_feature_names, index=X_train_copy.index)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=onehot_feature_names, index=X_test_copy.index)


X_train_copy = pd.concat([X_train_copy.drop(columns=nominal_features), X_train_onehot_df], axis=1)
X_test_copy = pd.concat([X_test_copy.drop(columns=nominal_features), X_test_onehot_df], axis=1)


#Train Logistic Regression

In [21]:
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train_copy, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [22]:
y_pred = model.predict(X_test_copy)

In [31]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9070971918750506

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95     10948
           1       0.67      0.36      0.47      1409

    accuracy                           0.91     12357
   macro avg       0.80      0.67      0.71     12357
weighted avg       0.89      0.91      0.89     12357



###**What are the accuracy, average precision, average recall, and average f1-score?**
* The accuracy is 0.91
* The average precision
  * macro avg = 0.80
  * Weighted avg = 0.89
* average recall
  * macro avg = 0.67
  * Weighted avg = 0.91
* average f1-score
  * macro avg = 0.71
  * Weighted avg = 0.89

## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [24]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [25]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [26]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [27]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [28]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

Number of training documents: 11314
Number of test documents: 7532
Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Explore Sample Document

In [29]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [30]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

SyntaxError: invalid syntax (ipython-input-2984329282.py, line 11)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


